# Demo of RtDeep

We define a simple network using RtDeep with microcircuits.

Task: mapping a 1-D signal in time onto another 1-D signal.

The model uses real time dynamics with Euler integration, LE, dendritic error propagation and a phased implementation of DTP-DRL.

## Definitions

In [1]:
%matplotlib notebook

import numpy as np
import pylab as plt
plt.rc('text', usetex=True)
plt.rc('font', size=18,family='serif')
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FormatStrFormatter
from microcircuit import *
import sys


#load autoreload, which automatically reloads the microcircuit.py upon execution
%reload_ext autoreload
%autoreload 1
%aimport microcircuit

In [2]:
# simulation settings

dt = 0.01 # in ms
tauxi = 10 * dt
Tpres = 100 * dt

In [151]:
# Init model: weights, voltages

layers = [1, 2, 3, 4]

# conductances
gl = 0.03
gbas = 0.1
gapi = 0.06
gden = 0.1
gnI = 0.06
gntgt = 0.06

# initialise voltages
uP_init = []
for i in range(1, len(layers)):
    uP_init.append(np.random.normal(0, 1, size=layers[i]))
    
uI_init = [np.random.normal(0, 1, size=layers[-1])]
    
# forward pp weights: connects all layers k, k+1
WPP_init = []
for i in range(len(layers)-1):
    WPP_init.append(np.random.normal(0, 1, size=(layers[i+1], layers[i])))

# p to inter: connects N-1 to N
WIP_init = [np.random.normal(0, 1, size=(layers[-1], layers[-2]))]

# backwards p to p: connects N to k
BPP_init = []
for i in range(len(layers)-1):
    BPP_init.append(np.random.normal(0, 1, size=(layers[i], layers[-1])))

# backwards inter to p: connects N to k
BPI_init = []
for i in range(len(layers)-1):
    BPI_init.append(np.random.normal(0, 1, size=(layers[i], layers[-1])))

In [152]:
# input rates: zero for now

r0 = np.zeros(shape=(int(Tpres / dt * 10), layers[0]))

In [153]:
mc1 = model(dt, tauxi, Tpres, 'FA', relu, d_relu, layers, uP_init, uI_init, WPP_init, WIP_init, BPP_init, BPI_init,
            gl=gl, gden=gden, gbas=gbas, gapi=gapi, gnI=gnI, gntgt=gntgt)

## Test self-predicting dynamics (no target)

In [154]:
mc1.set_self_predicting_state()

In [155]:
mc1.get_voltages()

([array([-0.66880624,  1.33312975]),
  array([-0.13684279, -1.00268295,  0.05522704]),
  array([-0.42855275, -0.32584572, -1.03219156,  0.76398363])],
 [array([ 1.94784151, -1.09901399,  1.46673554,  0.43653691])])

In [165]:
mc1.evolve_voltages()

In [161]:
u_time_series = []
for j in range(100):
    u_time_series.append(copy.deepcopy(mc1.get_voltages()))
    for i in range(100):
        mc1.evolve_voltages()

In [157]:
uI_time_series = [u[-1] for u in u_time_series]

In [158]:
# uI_time_series

In [164]:
mc1.uI_breve

[array([ 1.56865909e-08,  5.77463847e-08, -1.77713083e-07, -3.27326250e-07])]

In [160]:
mc1.uP

[array([-0.66858349,  1.33011869]),
 array([-0.13756484, -0.99931229,  0.05503083]),
 array([-0.42771921, -0.32523134, -1.03018931,  0.76253387])]